In [1]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

In [3]:
df=sns.load_dataset('tips')
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [5]:
#Handling missing values
#Handling categorical values
#handling ouliers
#feature scaling (not required in decision trees though)
#automating the entire model

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   total_bill  244 non-null    float64 
 1   tip         244 non-null    float64 
 2   sex         244 non-null    category
 3   smoker      244 non-null    category
 4   day         244 non-null    category
 5   time        244 non-null    category
 6   size        244 non-null    int64   
dtypes: category(4), float64(2), int64(1)
memory usage: 7.4 KB


In [11]:
df['time'].unique()

['Dinner', 'Lunch']
Categories (2, object): ['Lunch', 'Dinner']

In [12]:
from sklearn.preprocessing import LabelEncoder
encoder=LabelEncoder()
df['time']=encoder.fit_transform(df['time'])

In [15]:
df['time'].head()

0    0
1    0
2    0
3    0
4    0
Name: time, dtype: int64

In [16]:
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,0,2
1,10.34,1.66,Male,No,Sun,0,3
2,21.01,3.50,Male,No,Sun,0,3
3,23.68,3.31,Male,No,Sun,0,2
4,24.59,3.61,Female,No,Sun,0,4


In [19]:
df['time'].value_counts()

0    176
1     68
Name: time, dtype: int64

In [20]:
#independent and dependent features
X=df.drop('time',axis=1)
y=df['time']

In [22]:
X.head()

,total_bill,tip,sex,smoker,day,size
0,16.99,1.01,Female,No,Sun,2
1,10.34,1.66,Male,No,Sun,3
2,21.01,3.50,Male,No,Sun,3
3,23.68,3.31,Male,No,Sun,2
4,24.59,3.61,Female,No,Sun,4


In [23]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.25,random_state=30)

In [26]:
#CREATING PIPELINES NOW!!!

In [28]:
from sklearn.impute import SimpleImputer#Handling missing values
from sklearn.preprocessing import OneHotEncoder#Handling categorical values
from sklearn.preprocessing import StandardScaler#feature scaling (not required in decision trees though)
#automating the entire model

In [29]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer #To connect the pipeline we need to use this


In [30]:
categorical_cols=['sex','smoker','day']
numerical_cols=['total_bill','tip','size']

In [35]:
#Feature_Engineering Automation
cat_pipeline=Pipeline(
    steps=[
            ('imputer',SimpleImputer(strategy='most_frequent')),   ##replacing missing_values with mode
            ('onehotencoder',OneHotEncoder())
    ]
)

#for numerical_values
num_pipeline=Pipeline(
    steps=[
            ('imputer',SimpleImputer(strategy='median')),    #replacing na_values with mode
            ('scaler',StandardScaler())                     #feature_scaling
    ] 
)

In [37]:
preprocessor=ColumnTransformer([
    ('num_pipeline',num_pipeline,numerical_cols),    #pipleline_name, pipeline, cols_chosen_for_pipeline
    ('cat_pipeline',cat_pipeline,categorical_cols)
]
)

In [60]:
X_train=preprocessor.fit_transform(X_train)
X_test=preprocessor.transform(X_test)

In [73]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

In [67]:
#Automate Model Training Process
models={
        'Random Forest':RandomForestClassifier()}

In [74]:
models={
        'Random Forest':RandomForestClassifier(),
        'Decision Tree':DecisionTreeClassifier(),
        'SVC':SVC()}  #addinf another model

In [75]:
from sklearn.metrics import accuracy_score,classification_report

In [76]:
def evaluate_model(X_train,X_test,y_train,y_test,models):
    report={}
    for i in range(len(models)):
        model=list(models.values())[i]
        #Train Model
        model.fit(X_train,y_train)
        
    #predicitng testing data
        y_test_pred=model.predict(X_test)
        
    #get accuracy_score
        score=accuracy_score(y_test,y_test_pred)
        report[list(models.keys())[i]]=score
    return report

In [77]:
evaluate_model(X_train,X_test,y_train,y_test,models)

{'Random Forest': 0.9344262295081968,
 'Decision Tree': 0.9344262295081968,
 'SVC': 0.9508196721311475}

In [78]:
#HYPER_PARAMETER TUNING

In [81]:
classifier=RandomForestClassifier()

In [85]:
params={
        'max_depth':[3,5,10,None],
        'n_estimators':[100,200,300],
        'criterion':['gini','entropy']}

In [86]:
from sklearn.model_selection import RandomizedSearchCV
cv=RandomizedSearchCV(classifier,param_distributions=params,scoring='accuracy',cv=3,verbose=3)
cv.fit(X_train,y_train)

Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV 1/3] END criterion=entropy, max_depth=10, n_estimators=100;, score=0.951 total time=   0.2s
[CV 2/3] END criterion=entropy, max_depth=10, n_estimators=100;, score=0.984 total time=   0.2s
[CV 3/3] END criterion=entropy, max_depth=10, n_estimators=100;, score=0.967 total time=   0.2s
[CV 1/3] END criterion=entropy, max_depth=5, n_estimators=300;, score=0.967 total time=   0.5s
[CV 2/3] END criterion=entropy, max_depth=5, n_estimators=300;, score=0.984 total time=   0.5s
[CV 3/3] END criterion=entropy, max_depth=5, n_estimators=300;, score=0.967 total time=   0.5s
[CV 1/3] END criterion=entropy, max_depth=5, n_estimators=100;, score=0.967 total time=   0.2s
[CV 2/3] END criterion=entropy, max_depth=5, n_estimators=100;, score=0.984 total time=   0.2s
[CV 3/3] END criterion=entropy, max_depth=5, n_estimators=100;, score=0.967 total time=   0.2s
[CV 1/3] END criterion=entropy, max_depth=None, n_estimators=200;, score=0.951 to

RandomizedSearchCV(cv=3, estimator=RandomForestClassifier(),
                   param_distributions={'criterion': ['gini', 'entropy'],
                                        'max_depth': [3, 5, 10, None],
                                        'n_estimators': [100, 200, 300]},
                   scoring='accuracy', verbose=3)

In [87]:
cv.best_params_

{'n_estimators': 300, 'max_depth': 3, 'criterion': 'gini'}

In [95]:
y_pred1=cv.predict(X_test)

In [96]:
print(accuracy_score(y_pred1,y_test))

0.9508196721311475
